# HMM phi tuyến + Particle Filter cải tiến
Áp dụng cho chuỗi tiêu thụ điện năng
Dataset: Household Electric Power Consumption

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


In [ ]:
df = pd.read_csv('household_power_consumption.txt', sep=';', parse_dates={'Datetime': ['Date', 'Time']},
                 infer_datetime_format=True, na_values='?', low_memory=False)

df = df[['Datetime', 'Global_active_power']].dropna()
df['Global_active_power'] = df['Global_active_power'].astype(float)
df = df[df['Datetime'] < '2007-01-08']
df.reset_index(drop=True, inplace=True)

X = df['Global_active_power'].values
plt.plot(df['Datetime'], X)
plt.title("Chuỗi thời gian công suất tiêu thụ")
plt.xlabel("Thời gian")
plt.ylabel("Công suất (kW)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Improved Particle Filter cho HMM phi tuyến
class ImprovedParticleFilter:
    def __init__(self, n_particles, state_transition_fn, obs_likelihood_fn, resample_threshold=0.5):
        self.n_particles = n_particles
        self.state_transition_fn = state_transition_fn
        self.obs_likelihood_fn = obs_likelihood_fn
        self.resample_threshold = resample_threshold
        self.particles = None
        self.weights = None

    def initialize(self, init_fn):
        self.particles = init_fn(self.n_particles)
        self.weights = np.ones(self.n_particles) / self.n_particles

    def effective_sample_size(self):
        return 1. / np.sum(self.weights ** 2)

    def resample(self):
        indices = np.random.choice(self.n_particles, size=self.n_particles, p=self.weights)
        self.particles = self.particles[indices]
        self.weights.fill(1.0 / self.n_particles)
        self.particles += np.random.normal(0, 0.01, size=self.n_particles)  # regularization

    def update(self, observation):
        # Dự đoán bước tiếp theo
        self.particles = self.state_transition_fn(self.particles)

        # Cập nhật trọng số theo xác suất quan sát
        likelihoods = self.obs_likelihood_fn(self.particles, observation)
        self.weights *= likelihoods
        self.weights += 1e-300  # tránh chia cho 0
        self.weights /= np.sum(self.weights)

        if self.effective_sample_size() < self.resample_threshold * self.n_particles:
            self.resample()

        return np.average(self.particles, weights=self.weights)


In [ ]:
# Hàm chuyển trạng thái: x_t = x_{t-1} + noise
def state_transition(particles):
    return particles + np.random.normal(0, 0.05, size=particles.shape)

# Hàm likelihood: Gaussian centered tại observation
def obs_likelihood(particles, obs):
    return np.exp(-0.5 * ((particles - obs)**2) / 0.1**2)

# Khởi tạo
ipf = ImprovedParticleFilter(
    n_particles=500,
    state_transition_fn=state_transition,
    obs_likelihood_fn=obs_likelihood_fn,
    resample_threshold=0.5
)

ipf.initialize(lambda N: np.random.normal(loc=X[0], scale=0.5, size=N))

estimates = []

for obs in tqdm(X):
    estimate = ipf.update(obs)
    estimates.append(estimate)

# Vẽ kết quả
plt.figure(figsize=(15,5))
plt.plot(df['Datetime'], X, label='Quan sát thực tế', alpha=0.6)
plt.plot(df['Datetime'], estimates, label='Ước lượng (Particle Filter)', linewidth=2)
plt.title("Ước lượng chuỗi thời gian bằng Improved Particle Filter")
plt.xlabel("Thời gian")
plt.ylabel("Công suất (kW)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
